# Coursework 2



In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# Basic helper functions
def percent(x): return x / 100
def seed(n): return np.random.default_rng(n)

The `monte_carlo(μ, σ, price, days, iterations, rng)` function below simulates a share price defined by the equation $\frac{P_t}{P_{t-1}}=Z$ where $P_t$ is the price at time $t$ and $Z$ is a random draw from a normal distribution with a mean passed as $\mu$ and a standard deviation passed as $\sigma$.
_price_ is the initial price of the share on day 1, _days_ is the number of days to run the simulation for, _iterations_ is the number of times to repeat the simulation, and _rng_ is a [`numpy.random.Generator`](https://numpy.org/doc/stable/reference/random/generator) to supply the random numbers (e.g. if you need a fixed seed).

In [ ]:
def monte_carlo(μ=0, σ=1, price=100, days=252, iterations=1, rng=None):
    μ = percent(np.atleast_1d(μ))
    σ = percent(np.atleast_1d(σ))
    variations = max(μ.size, σ.size)
    if rng is None:
        rng = seed(None)
    shape = (days, variations, iterations)
    Z = np.empty(shape)
    for i in range(variations):
        Z[:,i,] = rng.normal(loc=1+μ[i%μ.size], scale=σ[i%σ.size], size=shape[::2])
    P = np.empty(shape)
    P[0,...] = price
    for i in range(1, days):
        P[i,...] = P[i-1,...] * Z[i,...]
    return np.squeeze(P)

These two small functions allow the calculation of profit from a call or put option on a whole array at once. _Pt_ can be a scalar or an array, whereas _X_ should be a scalar.

In [ ]:
def call_option(X, Pt):
    return np.maximum(X - Pt, 0)

def put_option(X, Pt):
    return np.maximum(Pt - X, 0)

In [ ]:
P = monte_carlo(μ=(0.1, 0.2, 0.3), σ=(1, 4, 6), days=756, rng=seed(345))

figure, plot = plt.subplots()
figure.set_size_inches(12, 9)
plot.plot(P[...,0], label='Company 1')
plot.plot(P[...,1], label='Company 2')
plot.plot(P[...,2], label='Company 3')
plot.set_title('756-day simulation of 3 different companies')
plot.set_xlabel('Days')
plot.set_ylabel('Projected Price')
plot.legend()

In [ ]:
def question2_mean(μ=0.05, X=175):
    P = monte_carlo(μ, σ=0.1, days=756, iterations=10_000, rng=seed(678))[-1]  # Slice off just the final price value
    return np.mean(call_option(X, P))